# 네이버 매장 -> 매장에 있는 review

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException
from tqdm import tqdm
from bs4 import BeautifulSoup

import time
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import requests
from openpyxl import Workbook
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# 정규 표현식으로 이모티콘, 특수문자 제거 (단, ?와 !는 제외)
def remove_special_characters(text):
    pattern = r'[^\w\s!?]|_'
    result = re.sub(pattern, '', text)
    return result

In [ ]:
# 데이터 불러오기
original_res_df = pd.read_excel('combined_file.xlsx')
original_df = original_res_df.copy()
original_df = original_df.loc[original_df['검색여부'] == '가능'].reset_index(drop=True)
original_df.drop('Unnamed: 0', axis=1, inplace=True)


# 컬럼 순서 바꾸기 -> 보기 좋으라고 바꿈
original_df['검색어'] = original_df['상세주소'] + " " + original_df['사업장명']
column_order = ['검색어', '업태구분명', '사업장명', '사업장명_원본', '시도', '지역구', '상세주소', '소재지전체주소', '도로명전체주소', '좌표정보(x)', '좌표정보(y)', '검색여부', '영업상태명']
original_df = original_df[column_order]
original_df.tail()

### 리뷰 데이터 크롤링

In [ ]:
start_num = 0 # 할 차례
num = 2
end_num = start_num + num
res_df = original_df[start_num:end_num]
new_res_df_test = res_df.reset_index(drop=True)
new_res_df_test

# 음식점 리뷰 크롤링

In [ ]:
# 빈 리스트 생성
user_id_list = []
content_list = []
date_list = []  
revisit_list = []
tag_category_name_list = []
restaurant_name_list = []
review_category_name_list = []

driver = webdriver.Chrome(ChromeDriverManager().install())

scroll_num = 5

for i in range(0,len(new_res_df_test)):
  
    name = new_res_df_test['검색어'][i]
    driver.get('https://map.naver.com/p/search/{}'.format(name))

    time.sleep(3)  
    try :
        if driver.find_elements(By.ID,'entryIframe') :
            entryIframe = driver.find_element(By.ID,'entryIframe')
            driver.switch_to.frame(entryIframe)
    except :
        pass 

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try : 
        # '리뷰' 탭의 href 속성 가져오기
        review_tab_href = soup.find('a', {'class': 'tpj9w _tab-menu', 'aria-selected': 'true'}).get('href')
        review_url = 'https://pcmap.place.naver.com'+review_tab_href

        driver.get(review_url)
        time.sleep(2.2)  

    except :
        print('끝')

    # 현재 페이지 URL 가져오기
    current_url = driver.current_url

    try : 
        # 리뷰 url로 이동 
        modified_url = current_url.replace('/home', '/review/visitor')
        driver.get(modified_url)
    except :
        continue

    # 리뷰 스크래핑 시작
    try:
        # 스크롤
        for _ in range(scroll_num):
            current_scroll_position = driver.execute_script('return window.scrollY;') # 초기 스크롤 위치 

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) 
            
            # 'TeItc' 클래스를 가진 요소가 클릭 가능할 때까지 기다리고, 보이도록 스크롤
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'TeItc')))
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # 더보기 클릭
            button = driver.find_element_by_class_name('fvwqf')
            button.click()

            # 'fvwqf' 클래스를 가진 요소가 나타날 때까지 스크롤
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.click()

            # ------------ 리뷰, 태그 '내용 더보기' 버튼 클릭 ----------------------
            review_buttons = driver.find_elements_by_class_name('xHaT3')  
            for button1 in review_buttons:
                try:
                    button1.click()
                except :
                    pass

            tag_buttons = driver.find_elements_by_css_selector('.gyAGI a.P1zUJ.ZGKcF')  
            for button2 in tag_buttons:
                try:
                    button2.click()
                except :
                    pass
            # ----------------------------------------------
            time.sleep(3)

            # ---------------스크롤위치가 변하지 않으면 리뷰를 다 크롤링한 것이니까 탈출 -------------------------
            # 현재 스크롤 
            new_scroll_position = driver.execute_script('return window.scrollY;')

            # 현재 위치와 이전 위치가 같으면 반복문 탈출
            if new_scroll_position == current_scroll_position:
                break

            # 이전 스크롤 위치 업데이트
            prev_scroll_position = current_scroll_position
            # -------------------------------------------------------------------------------------------------
            
            html = driver.page_source
            soup = BeautifulSoup(html, 'lxml')
            reviews = soup.select('li.owAeM') 

        # 리뷰 크롤링 시작
        for r in reviews:
            user_id = r.select_one('div.qgLL3 span.P9EZi')
            review_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(1)')
            photo_count = r.select_one('div.Gt2_9 span.RNn6x:nth-of-type(2)')
            
            content = r.select_one('div.vg7Fp span.zPfVt')
            
            try:
                tag = r.select_one('div.ERkm0 span.sIv5s')
            except:
                tag = ""
            
            date = r.select_one('div.jxc2b div.D40bm span.CKUdu time')
            revisit = r.select_one('div.jxc2b div.D40bm span.CKUdu:nth-of-type(2)')
            
            # 없는 경우 
            user_id_text = user_id.text if user_id else ''
            review_count_text = review_count.text if review_count else ''
            photo_count_text = photo_count.text if photo_count else ''
            content_text = content.text if content else ''
            tag_text = tag.text if tag else ''
            date_text = date.text if date else ''
            revisit_text = revisit.text.replace('번째 방문', '') if revisit else ''
            
            # 리스트에 합치기
            restaurant_name_list.append(name)
            user_id_list.append(user_id_text)
            review_category_name_list.append(tag_text)
            date_list.append(date_text)
            revisit_list.append(revisit_text)
            content_list.append(remove_special_characters(content_text))

    except Exception as e:
        print(e)


# 데이터프레임 얹히기
review_data_list = {
    '검색어': restaurant_name_list,
    'user_id': user_id_list,
    'review': content_list,
    'date': date_list,
    'revisit': revisit_list,
    'tag': review_category_name_list
}

# 리뷰 데이터 프레임 완성
review_result_df = pd.DataFrame(review_data_list)

review_result_df

In [ ]:
review_result_df.to_excel(f'./data/restaurant_review_data/restaurant_review_df_{start_num}_{end_num}.xlsx', index=False)

In [ ]:
# review_result_df.to_excel('.data/음식점_리뷰데이터.xlsx')
# tag_result_df.to_excel('.data/음식점_태그데이터.xlsx')

In [ ]:
import re

def remove_special_characters(text):
    # 정규 표현식으로 이모티콘, 특수문자, 아스키코드 제거 (단, ?와 !는 제외)
    pattern = r'[^\w\s!?]|_'
    result = re.sub(pattern, '', text)
    return result


text = "Hello, World! 😊 How are you today??????!!!? #Python"
result = remove_special_characters(text)
print(result)